In [1]:
##### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import sklearn

In [3]:
#seed 고정
torch.manual_seed(1)

In [4]:
# data로딩
train = pd.read_csv("/kaggle/input/2021-ai-quiz1-p1/train.csv")
test = pd.read_csv("/kaggle/input/2021-ai-quiz1-p1/test.csv")
submit = pd.read_csv("/kaggle/input/2021-ai-quiz1-p1/submit_sample.csv")

In [5]:
# 데이터 확인
print(train.head())
print(test.head())
print(submit.head())

In [6]:
# 데이터 텐서에 올리기 ( 필요없는 컬럼 제외)
x_train = np.array(train.drop(['avgPrice','year'],axis=1))
y_train = np.array(train['avgPrice'])
x_test = np.array(test.drop(['year'],axis=1))

x_train = torch.Tensor(x_train)
y_train = torch.Tensor(y_train).unsqueeze(1)
x_test = torch.Tensor(x_test)

print(x_train.shape, y_train.shape)

In [7]:
# 학습 설정
w = torch.zeros((4,1),requires_grad= True)
b = torch.zeros(1,requires_grad =True)
optim = torch.optim.SGD([w,b],lr=1e-4)
epochs = 1000
loss = torch.nn.MSELoss()

In [8]:
# 학습
for epoch in range(epochs+1):
    optim.zero_grad()
    
    h = x_train.matmul(w)+b
    cost = loss(h,y_train)
    
    cost.backward()
    optim.step()
    if epoch % 100 == 0:
        print(cost.item())

In [9]:
# 학습에 관련없이 예측값 확인
with torch.no_grad():
    predict = x_test.matmul(w) + b
submit['Expected'] = predict

In [10]:
submit.to_csv("submission.csv",index=False)

try 
1: lr = 1e-5 cost가 잘 감소하지 않음
2: lr = 1e-3 cost가 감소함 -> eppch를 늘려서 테스트 해보니 2173509.0 정도에 고정됨 
잘 학습되었다고 판단